# Stratified sampling
Stratified sampling is a method for sampling subpopulations (strata) independently. It is commonly used to reduce sampling error by ensuring that subgroups are adequately represented in the sample.

Stratified sampling was added in MADlib 1.12.

In [9]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [10]:
# Greenplum Database 5.4.0 on GCP (demo machine)
#%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpdbchina@madlib'

In [11]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rel/v1.11-23-gfdf7b6d, cmake configuration time: Wed Jun 28 18:06:35 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create input table

In [15]:
%%sql 
DROP TABLE IF EXISTS test;

CREATE TABLE test(
    id1 INTEGER,
    id2 INTEGER,
    gr1 INTEGER,
    gr2 INTEGER
);

INSERT INTO test VALUES
(1,0,1,1),
(2,0,1,1),
(3,0,1,1),
(4,0,1,1),
(5,0,1,1),
(6,0,1,1),
(7,0,1,1),
(8,0,1,1),
(9,0,1,1),
(9,0,1,1),
(9,0,1,1),
(9,0,1,1),
(0,1,1,2),
(0,2,1,2),
(0,3,1,2),
(0,4,1,2),
(0,5,1,2),
(0,6,1,2),
(10,10,2,2),
(20,20,2,2),
(30,30,2,2),
(40,40,2,2),
(50,50,2,2),
(60,60,2,2),
(70,70,2,2);

SELECT * FROM test ORDER BY gr1, gr2, id1, id2;

Done.
Done.
25 rows affected.
25 rows affected.


id1,id2,gr1,gr2
1,0,1,1
2,0,1,1
3,0,1,1
4,0,1,1
5,0,1,1
6,0,1,1
7,0,1,1
8,0,1,1
9,0,1,1
9,0,1,1


# 2.  Sample without replacement

In [16]:
%%sql
DROP TABLE IF EXISTS out;

SELECT madlib.stratified_sample(
                                'test',    -- Source table
                                'out',     -- Output table
                                0.5,       -- Sample proportion
                                'gr1,gr2', -- Strata definition
                                'id1,id2', -- Columns to output
                                FALSE);    -- Sample without replacement

SELECT * FROM out ORDER BY gr1,gr2,id1,id2;

Done.
1 rows affected.
13 rows affected.


gr1,gr2,id1,id2
1,1,1,0
1,1,2,0
1,1,3,0
1,1,6,0
1,1,9,0
1,1,9,0
1,2,0,1
1,2,0,2
1,2,0,5
2,2,30,30


# 3.  Sample with replacement

In [71]:
%%sql
DROP TABLE IF EXISTS out;

SELECT madlib.stratified_sample(
                                'test',    -- Source table
                                'out',     -- Output table
                                0.5,       -- Sample proportion
                                'gr1,gr2', -- Strata definition
                                'id1,id2', -- Columns to output
                                TRUE);     -- Sample with replacement

SELECT * FROM out ORDER BY gr1,gr2,id1,id2;

Done.
1 rows affected.
13 rows affected.


gr1,gr2,id1,id2
1,1,1,0
1,1,2,0
1,1,3,0
1,1,4,0
1,1,8,0
1,1,9,0
1,2,0,5
1,2,0,5
1,2,0,6
2,2,20,20
